# Download data

You can compose queries to download raw data.

In [ ]:
import getpass
import platform

import matplotlib.pyplot as plt
import pandas as pd

import doplaydo.dodata as dd

username = getpass.getuser()

PROJECT_ID = f"cutback-{username}-{platform.node()}"

You have access to:

- dd.Project
- dd.Die
- dd.Wafer
- dd.ParentCell
- dd.Cell
- dd.Device
- dd.DeviceData

Where each model has its table columns as attributes.


You can use `get_data_by_query` to query a subset of data filtered by a list of clauses.

It will return a list of tuples where the first element is a `DeviceData` object and the second one is a pandas DataFrame.

To retrieve specific data, we’ll use the GetDataByQuery function, which allows us to filter data by a list of clauses. In this example, we’ll filter by project ID. 

The GetDataByQuery function lets you retrieve data based on a list of specified filters. For example, here we filter by project ID and device ID with a specific name to obtain relevant device data. Part of this data is stored in a Pandas DataFrame with columns such as wavelength and output power. Plotting this data gives us a spectrum of wavelength vs. output power.



In [ ]:
with dd.get_session() as session:
    data_tuples = dd.get_data_by_query(
        [dd.Project.project_id == PROJECT_ID], limit=1, session=session
    )
    device_data, df = data_tuples[0]  # each tuple has DeviceData and pandas Dataframe
    device_id = device_data.device.device_id
device_data

In [ ]:
device_id

In [ ]:
df

In [ ]:
plt.plot(df["wavelength"], df["output_power"])
plt.xlabel("wavelength (nm)")
plt.ylabel("output power (dBm)")
plt.title(device_id)

You can aggregate the dataframes and the device data objects

In [ ]:
dfs = [dt[1] for dt in data_tuples]  # dataframes
dds = [dt[0] for dt in data_tuples]  # device data objects

The DeviceData object allows you to navigate the data model and access related fields across the hierarchy.

You can traverse from DeviceData to other tables by following the dashed arrows in the data structure, with each column representing an attribute of the corresponding table.

The database is organized into several hierarchical tables to streamline data relationships:

- Projects: Each chip is assigned to a specific project.
- Cells: Cells represent entities within a project.
- Devices: Each cell instance corresponds to a device with defined X, Y coordinates.
- Device_Data: Stores testing or simulation data for each device.
- Analysis: Device data, dies, or wafers can be analyzed using specific analysis functions.

For each analysis, we track details such as the originating function, associated die, and wafer, ensuring a clear, structured relationship within the data. Additionally, wafers link back to projects, and dies are connected to their corresponding wafers.

![](https://i.imgur.com/CfOHl57.png)


We can also examine additional details, like the length of the data tuples and all metadata related to the device, including device ID, die origin (x, y coordinates), wafer ID, set ID, parent cell, and the associated project.

In [ ]:
with dd.get_session() as session:
    data_tuples = dd.get_data_by_query(
        [dd.Project.project_id == PROJECT_ID], limit=1, session=session
    )
    dfs = [dt[1] for dt in data_tuples]  # dataframes
    dds = [dt[0] for dt in data_tuples]  # device data objects
    print("device id: ", dds[0].device.device_id)
    print("die x: ", dds[0].die.x)
    print("die y: ", dds[0].die.y)
    print("wafer id: ", dds[0].die.wafer.wafer_id)
    print("cell id: ", dds[0].device.cell.cell_id)
    print("parent cell id: ", dds[0].device.parent_cell.cell_id)
    print("project id: ", dds[0].device.cell.project.project_id)

For example, you can access the Cell table and all its columns:

In [ ]:
with dd.get_session() as session:
    data_tuples = dd.get_data_by_query(
        [dd.Project.project_id == PROJECT_ID], limit=1, session=session
    )
    dfs = [dt[1] for dt in data_tuples]  # dataframes
    dds = [dt[0] for dt in data_tuples]  # device data objects
    print(dds[0].device.cell)

## Build table for JMP

You can build a flat table by combining all the device data and pandas dataframes.  

In [ ]:
with dd.get_session() as session:
    data_tuples = dd.get_data_by_query(
        [
            dd.Project.project_id == PROJECT_ID,
            dd.Device.device_id == device_id,  # only get data for this device
            dd.Die.x == 0,
            dd.Die.y == 0,
        ],
        session=session,
    )
    len(data_tuples)
    dds = [dt[0] for dt in data_tuples]  # device data objects
    dfs = [dt[1] for dt in data_tuples]  # dataframes
dfs[0]

In [ ]:
with dd.get_session() as session:
    data_tuples = dd.get_data_by_query(
        [
            dd.Project.project_id == PROJECT_ID,
            dd.Device.device_id == device_id,  # only get data for this device
            dd.Die.x == 0,
            dd.Die.y == 0,
        ],
        session=session,
    )
    len(data_tuples)
    dds = [dt[0] for dt in data_tuples]  # device data objects
    dfs = [dt[1] for dt in data_tuples]  # dataframes
    dfs_all = []

    for device_data, df in zip(dds, dfs, strict=False):
        df["device_id"] = device_data.device.device_id
        df["die_x"] = device_data.die.x
        df["die_y"] = device_data.die.y
        df["wafer_id"] = device_data.die.wafer.wafer_id
        df["cell_id"] = device_data.device.cell.cell_id
        df["parent_cell_id"] = device_data.device.parent_cell.cell_id
        dfs_all.append(df)

In [ ]:
dfs_all = pd.concat(dfs_all)  # You can concatenate all dataFrames together
dfs_all

## Advanced queries


To build advanced queries to filter metadata you can use the `attribute_filter` method. You can also use `or_`, `and_` for conditional clauses.

### Conditional filter

In [ ]:
with dd.get_session() as session:
    data_tuples = dd.get_data_by_query(
        [
            dd.Project.project_id == PROJECT_ID,
            dd.Device.device_id == device_id,  # only get data for this device
        ],
        session=session,
    )
len(data_tuples)

By default anything you pass to the list joins the clauses as `and_`

In [ ]:
with dd.get_session() as session:
    data_tuples = dd.get_data_by_query(
        [
            dd.and_(
                dd.Project.project_id == PROJECT_ID,
                dd.Device.device_id == device_id,  # only get data for this device
            )
        ],
        session=session,
    )
len(data_tuples)

You can also use an OR condition.

The query below filters by project id `PROJECT_ID` AND either device id

Because there are 45 measurements per device, one for each die, if you look for two specific devices, you will get 90 measurements.

In [ ]:
device_id2 = "cutback_cutback_loss_1_529485_116311"

with dd.get_session() as session:
    data_tuples = dd.get_data_by_query(
        [
            dd.Project.project_id == PROJECT_ID,
            dd.or_(dd.Device.device_id == device_id, dd.Device.device_id == device_id2),
        ],
        session=session,
    )
len(data_tuples)

You can also combine conditionals.

In the example below, you can get all the data for the specified project id, device id and either die coordinate.

In [ ]:
with dd.get_session() as session:
    data_tuples = dd.get_data_by_query(
        [
            dd.Project.project_id == PROJECT_ID,
            dd.Device.device_id == device_id,  # only get data for this device
            dd.or_(
                dd.and_(dd.Die.x == 1, dd.Die.y == 1),  # either die 1,1
                dd.and_(dd.Die.x == 0, dd.Die.y == 0),  # or die 0,0
            ),
        ],
        session=session,
    )
len(data_tuples)

### Attribute filter

You can filter attributes of any data models by passing a list of conditions as shown below.

You can use `dd.Cell/dd.Wafer/...`, `key`, `value`

You can only filter for values that are str, bool, int, float.

The query below filters Cells with 400 components in the cutback that are in a particular project and die coordinates.

In [ ]:
with dd.get_session() as session:
    device_data_objects = dd.get_data_by_query(
        [
            dd.attribute_filter(
                dd.Cell, "components", 400
            ),  # checking for values in JSON attributes
            dd.Project.project_id == PROJECT_ID,
            dd.Die.x == 0,
            dd.Die.y == 0,
        ],
        session=session,
    )

In [ ]:
len(device_data_objects)

You can combine conditional and attribute filter clauses together.

You can get the data for Cells for a given project with 400 or 816 components in the cutback

In [ ]:
with dd.get_session() as session:
    data_tuples = dd.get_data_by_query(
        [
            dd.Project.project_id == PROJECT_ID,
            # dd.attribute_filter(dd.Cell, "components", 400),
            dd.or_(
                dd.attribute_filter(dd.Cell, "components", 400),
                dd.attribute_filter(dd.Cell, "components", 816),
            ),
        ],
        session=session,
    )
len(data_tuples)